## 仅供交叉验证 前馈神经网络（NNAR）-按趋势分类

In [1]:
import pickle
import numpy as np
import random
from utils import *
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
n_input = 11

读取数据

In [2]:
# gene_arr_path = r'../output/gene_editing/es_with_decay.array'
# transplant_arr_path = r'../output/transplant/es_with_decay.array'

# gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
# transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 17, 10)
Shape of the transplant array: (5141, 17, 10)


### 截断数据
2019年为无效数据

In [3]:
# gene_arr = gene_arr[:, :-1, :]
# transplant_arr = transplant_arr[:, :-1, :]

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


### 规范数据并获取5折交叉检验所需的训练集和验证集

In [4]:
# scaler, data = scale_data(transplant_arr, 'standard')

# # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
# X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2],transplant_arr[:, n_input, -1]
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 按趋势划分

In [2]:
def split_data_by_trend(data, targets):
    up_data = []
    down_data = []
    up_target = []
    down_target = []
    
    for i in range(len(data)):
        a, b = np.polyfit(range(len(data[i])), data[i, :, -2].reshape(-1), 1)
        if a > 0:
            up_data.append(data[i])
            up_target.append(targets[i])
        else:
            down_data.append(data[i])
            down_target.append(targets[i])
    return np.array(up_data), np.array(up_target), np.array(down_data), np.array(down_target)

### 构建模型

In [3]:
def root_mean_squared_error(y_true, y_pred):
        return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true), axis=-1)) 

# def build_direct_dnn_model():
#     model = keras.models.Sequential()
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dense(256, activation='relu'))
#     model.add(Dense(5))
    
#     optimizer=keras.optimizers.Adam(learning_rate=1e-4)
#     model.compile(loss=root_mean_squared_error, optimizer=optimizer)
#     return model
def build_direct_dnn_model(n_layers=2, n_units=256):
    model = keras.models.Sequential()
    model.add(Flatten())
    for i in range(n_layers):
        model.add(Dense(n_units, activation='relu'))
    model.add(Dense(5))
    
    optimizer=keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(loss=root_mean_squared_error, optimizer=optimizer)
    return model

### 进行训练和评估
使用EarlyStopping和Checkpoint做训练停止方式

In [4]:
def cross_validation(X, y, y_cat, kfold, scaler, n_layers, n_units):
    overall_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }

    annual_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }
    
    for train, test in kfold.split(X, y_cat):
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        models = []
        
        # 按总量划分数据集
        X_train1, y_train1, X_train2, y_train2 = split_data_by_trend(X_train, y_train)
        train_xs = [X_train1, X_train2]
        train_ys = [y_train1, y_train2]
        
        X_test1, y_test1, X_test2, y_test2 = split_data_by_trend(X_test, y_test)
        test_xs = [X_test1, X_test2]
        test_ys = [y_test1, y_test2]
        i_s = [1, 2]
        
        # 训练
        for i in range(len(i_s)):
            model = build_direct_dnn_model(n_layers, n_units)
            history = model.fit(train_xs[i], train_ys[i], epochs=300, batch_size=16, verbose=1, validation_data=(test_xs[i], test_ys[i]),
                           callbacks=[
                               EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto', restore_best_weights=True)
                           ])
            models.append(model)
        
        # 预测
        y_test = []
        y_pred = []
        for i in range(len(i_s)):
            y_test.append(test_ys[i])
            y_pred.append(models[i].predict(test_xs[i]).reshape(test_ys[i].shape))
        
        y_test = np.concatenate(y_test)
        y_pred = np.concatenate(y_pred)

        metrics = ['mae', 'rmse','ndcg', 'mape', 'r2', 'pearson', 'acc']
        for m in metrics:
            overall, annual = eval_model(m, y_test, y_pred, scaler)
            overall_metrics[m].append(overall)
            annual_metrics[m].append(annual)
    
    return overall_metrics, annual_metrics


In [5]:
def full_pipeline():
    gene_arr_path = r'../output/gene_editing/es_with_decay.array'
    transplant_arr_path = r'../output/transplant/es_with_decay.array'

    gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
    transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))
    
    gene_arr = gene_arr[:, :-1, :]
    transplant_arr = transplant_arr[:, :-1, :]

    print('Shape of the gene_editing array:',gene_arr.shape)
    print('Shape of the transplant array:',transplant_arr.shape)
    
    metrics = {
        'gene':{
            'overall':{},
            'annual':{}
        },
        'transplant':{
            'overall':{},
            'annual':{}
        }
    }
    
    for name, dataset in zip(['gene', 'transplant'], [gene_arr, transplant_arr]):
        scaler, data = scale_data(dataset, 'standard')

        # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
        X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2], dataset[:, n_input, -1]
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
#         overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler)
        if name == 'gene':
            overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler, 3, 128)
        elif name == 'transplant':
            overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler, 5, 256)
        
        for metric, value in overall_metrics.items():
            metrics[name]['overall'][metric] = np.mean(value)
        
        for metric, value in annual_metrics.items():
            metrics[name]['annual'][metric] = np.mean(np.array(value), axis=0)
    
    pickle.dump(metrics, open('mlp_metrics.dict', 'wb'))
    
    return metrics

In [6]:
metrics = full_pipeline()

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)
Train on 988 samples, validate on 256 samples
Epoch 1/300
988/988 [==============================] - 1s 1ms/sample - loss: 1.2336 - val_loss: 1.1642
Epoch 2/300
988/988 [==============================] - 0s 199us/sample - loss: 1.1086 - val_loss: 1.1015
Epoch 3/300
988/988 [==============================] - 0s 203us/sample - loss: 1.0722 - val_loss: 1.0859
Epoch 4/300
988/988 [==============================] - 0s 197us/sample - loss: 1.0604 - val_loss: 1.0798
Epoch 5/300
988/988 [==============================] - 0s 199us/sample - loss: 1.0541 - val_loss: 1.0770
Epoch 6/300
988/988 [==============================] - 0s 216us/sample - loss: 1.0499 - val_loss: 1.0745
Epoch 7/300
988/988 [==============================] - 0s 212us/sample - loss: 1.0470 - val_loss: 1.0750
Epoch 8/300
988/988 [==============================] - 0s 224us/sample - loss: 1.0433 - val_loss: 1.0750
Epoch 9/300
988/988 [=

1115/1115 [==============================] - 0s 267us/sample - loss: 0.7455 - val_loss: 0.7761
Epoch 10/300
1115/1115 [==============================] - 0s 265us/sample - loss: 0.7446 - val_loss: 0.7764
Epoch 11/300
1115/1115 [==============================] - 0s 243us/sample - loss: 0.7444 - val_loss: 0.7770
Epoch 12/300
1115/1115 [==============================] - 0s 265us/sample - loss: 0.7443 - val_loss: 0.7762
Epoch 13/300
1115/1115 [==============================] - 0s 248us/sample - loss: 0.7431 - val_loss: 0.7766
Epoch 14/300
1115/1115 [==============================] - 0s 264us/sample - loss: 0.7441 - val_loss: 0.7765
Train on 995 samples, validate on 249 samples
Epoch 1/300
995/995 [==============================] - 1s 966us/sample - loss: 1.2050 - val_loss: 1.1160
Epoch 2/300
995/995 [==============================] - 0s 250us/sample - loss: 1.0893 - val_loss: 1.0674
Epoch 3/300
995/995 [==============================] - 0s 259us/sample - loss: 1.0698 - val_loss: 1.0587
Epoc

1125/1125 [==============================] - 0s 248us/sample - loss: 0.7459 - val_loss: 0.7901
Epoch 3/300
1125/1125 [==============================] - 0s 256us/sample - loss: 0.7449 - val_loss: 0.7901
Epoch 4/300
1125/1125 [==============================] - 0s 263us/sample - loss: 0.7443 - val_loss: 0.7892
Epoch 5/300
1125/1125 [==============================] - 0s 247us/sample - loss: 0.7442 - val_loss: 0.7894
Epoch 6/300
1125/1125 [==============================] - 0s 243us/sample - loss: 0.7433 - val_loss: 0.7899
Epoch 7/300
1125/1125 [==============================] - 0s 245us/sample - loss: 0.7434 - val_loss: 0.7904
Epoch 8/300
1125/1125 [==============================] - 0s 246us/sample - loss: 0.7433 - val_loss: 0.7908
Epoch 9/300
1125/1125 [==============================] - 0s 246us/sample - loss: 0.7426 - val_loss: 0.7897
Epoch 10/300
1125/1125 [==============================] - 0s 277us/sample - loss: 0.7419 - val_loss: 0.7902
Epoch 11/300
1125/1125 [========================

1004/1004 [==============================] - 0s 251us/sample - loss: 0.9372 - val_loss: 1.0135
Epoch 64/300
1004/1004 [==============================] - 0s 252us/sample - loss: 0.9368 - val_loss: 1.0091
Epoch 65/300
1004/1004 [==============================] - 0s 246us/sample - loss: 0.9338 - val_loss: 1.0187
Epoch 66/300
1004/1004 [==============================] - 0s 266us/sample - loss: 0.9323 - val_loss: 1.0141
Epoch 67/300
1004/1004 [==============================] - 0s 243us/sample - loss: 0.9323 - val_loss: 1.0188
Epoch 68/300
1004/1004 [==============================] - 0s 255us/sample - loss: 0.9298 - val_loss: 1.0191
Epoch 69/300
1004/1004 [==============================] - 0s 246us/sample - loss: 0.9288 - val_loss: 1.0144
Epoch 70/300
1004/1004 [==============================] - 0s 250us/sample - loss: 0.9261 - val_loss: 1.0157
Epoch 71/300
1004/1004 [==============================] - 0s 251us/sample - loss: 0.9266 - val_loss: 1.0140
Epoch 72/300
1004/1004 [=================

580/580 [==============================] - 0s 382us/sample - loss: 0.3274 - val_loss: 0.4579
Epoch 15/300
580/580 [==============================] - 0s 382us/sample - loss: 0.3179 - val_loss: 0.4565
Epoch 16/300
580/580 [==============================] - 0s 383us/sample - loss: 0.3116 - val_loss: 0.4562
Epoch 17/300
580/580 [==============================] - 0s 380us/sample - loss: 0.3050 - val_loss: 0.4557
Epoch 18/300
580/580 [==============================] - 0s 385us/sample - loss: 0.3045 - val_loss: 0.4570
Epoch 19/300
580/580 [==============================] - 0s 444us/sample - loss: 0.2965 - val_loss: 0.4573
Epoch 20/300
580/580 [==============================] - 0s 375us/sample - loss: 0.2924 - val_loss: 0.4550
Epoch 21/300
580/580 [==============================] - 0s 418us/sample - loss: 0.2864 - val_loss: 0.4611
Epoch 22/300
580/580 [==============================] - 0s 370us/sample - loss: 0.2837 - val_loss: 0.4591
Epoch 23/300
580/580 [==============================] - 0s 

577/577 [==============================] - 0s 455us/sample - loss: 0.3757 - val_loss: 0.4098
Epoch 9/300
577/577 [==============================] - 0s 432us/sample - loss: 0.3668 - val_loss: 0.4099
Epoch 10/300
577/577 [==============================] - 0s 446us/sample - loss: 0.3606 - val_loss: 0.4112
Epoch 11/300
577/577 [==============================] - 0s 442us/sample - loss: 0.3555 - val_loss: 0.4110
Epoch 12/300
577/577 [==============================] - 0s 434us/sample - loss: 0.3495 - val_loss: 0.4112
Epoch 13/300
577/577 [==============================] - 0s 365us/sample - loss: 0.3436 - val_loss: 0.4155
Epoch 14/300
577/577 [==============================] - 0s 370us/sample - loss: 0.3375 - val_loss: 0.4139
Epoch 15/300
577/577 [==============================] - 0s 373us/sample - loss: 0.3349 - val_loss: 0.4159
Epoch 16/300
577/577 [==============================] - 0s 418us/sample - loss: 0.3270 - val_loss: 0.4136
Epoch 17/300
577/577 [==============================] - 0s 3

In [7]:
metrics

{'gene': {'overall': {'mae': 0.7291975904675908,
   'rmse': 1.2871031342506778,
   'ndcg': 0.4673787741071691,
   'mape': 4.422155277734545,
   'r2': 0.24244890230809224,
   'pearson': 0.524190051690691,
   'acc': 0.3317444264831185},
  'annual': {'mae': array([0.41338131, 0.57355205, 0.74987375, 0.88439481, 1.02478603]),
   'rmse': array([0.75304537, 0.95914567, 1.31467579, 1.48858523, 1.6651921 ]),
   'ndcg': array([0.4623395 , 0.43362874, 0.14620785, 0.19055866, 0.15106367]),
   'mape': array([3.42139588, 2.99739742, 3.24076012, 6.14907401, 6.30214895]),
   'r2': array([0.42885064, 0.29498014, 0.18880622, 0.05274659, 0.02198769]),
   'pearson': array([0.66843551, 0.56805361, 0.47478036, 0.32031706, 0.26847979]),
   'acc': array([0.58150882, 0.18466888, 0.23798611, 0.33935394, 0.31520438])}},
 'transplant': {'overall': {'mae': 0.7658888161691443,
   'rmse': 1.271241089721329,
   'ndcg': 0.4850424174440864,
   'mape': 3.8585934696807804,
   'r2': 0.42290244232094354,
   'pearson': 0.6

In [7]:
metrics

{'gene': {'overall': {'mae': 0.7297824076242991,
   'rmse': 1.288492394560198,
   'ndcg': 0.450425276583966,
   'mape': 4.379720797710389,
   'r2': 0.24096271379741033,
   'pearson': 0.5239841606169792,
   'acc': 0.33765443023983865},
  'annual': {'mae': array([0.41629491, 0.57294854, 0.75242516, 0.88069069, 1.02655274]),
   'rmse': array([0.75623629, 0.96455199, 1.31755807, 1.48652634, 1.66621578]),
   'ndcg': array([0.42948659, 0.36199947, 0.15691518, 0.21310819, 0.14353297]),
   'mape': array([3.27338697, 2.92294838, 3.34444352, 6.13713268, 6.22069244]),
   'r2': array([0.42536017, 0.28595323, 0.18519127, 0.05494907, 0.02053747]),
   'pearson': array([0.66756681, 0.55964269, 0.47305132, 0.33346943, 0.2686703 ]),
   'acc': array([0.57891309, 0.20543948, 0.25955221, 0.3371119 , 0.30725548])}},
 'transplant': {'overall': {'mae': 0.7741190799377897,
   'rmse': 1.2768263814311376,
   'ndcg': 0.521502225186101,
   'mape': 3.5752431968110856,
   'r2': 0.4178778432693182,
   'pearson': 0.65